In [4]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv('C:/Users/inho0/Downloads/confer/data_set/final.csv')
data['EPDO_cnt'] = 12 * data['dth_dnv_cnt'] + 6 * data['se_dnv_cnt'] + 3 * data['sl_dnv_cnt']
data.head()

X = data[['bus_station_cnt', 'traffic_cnt', 'pleasure_cnt', 'seoul_police_cnt', 'park_cnt', 'crossroad_cnt',
          '도로10이하_cnt', '도로10_20_cnt', '도로20_30_cnt', '도로30이상_cnt', 'child_cnt', 'silver_cnt', 'crosswalk_cnt']]
y = data['EPDO_cnt']

X = sm.add_constant(X)

# 포아송 회귀 모델 적용
poisson_model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

print(poisson_model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:               EPDO_cnt   No. Observations:                 1200
Model:                            GLM   Df Residuals:                     1186
Model Family:                 Poisson   Df Model:                           13
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -13033.
Date:                Thu, 11 Jul 2024   Deviance:                       25058.
Time:                        09:39:06   Pearson chi2:                 6.74e+04
No. Iterations:                     8   Pseudo R-squ. (CS):             0.9956
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0952      0.040  

In [11]:
# 과산포 확인
mean_caslt = np.mean(data['EPDO_cnt'])
var_caslt = np.var(data['EPDO_cnt'])
print(f'평균: {mean_caslt}, 분산: {var_caslt}')

if var_caslt > mean_caslt:
    print('과산포가 존재할 가능성이 있습니다.')

# 포아송 회귀 모델 적합
poisson_model = smf.glm(formula='EPDO_cnt ~ bus_station_cnt + traffic_cnt + pleasure_cnt + seoul_police_cnt + park_cnt + crossroad_cnt + 도로10이하_cnt + 도로10_20_cnt + 도로20_30_cnt + 도로30이상_cnt + child_cnt + silver_cnt + crosswalk_cnt', 
                        data=data, 
                        family=sm.families.Poisson()).fit()

pearson_chi2 = sum(poisson_model.resid_pearson**2)
df_resid = poisson_model.df_resid
dispersion = pearson_chi2 / df_resid
print(f'Pearson chi2: {pearson_chi2}')
print(f'Degree of freedom: {df_resid}')
print(f'Dispersion: {dispersion}')

if dispersion > 1:
    print('과산포가 존재합니다. 음이항 회귀 모델을 고려해야 합니다.')
else:
    print('과산포가 존재하지 않습니다. 포아송 회귀 모델이 적합할 수 있습니다.')


평균: 6.2725, 분산: 411.57824375000007
과산포가 존재할 가능성이 있습니다.
Pearson chi2: 67394.84041978889
Degree of freedom: 1186
Dispersion: 56.82532919037849
과산포가 존재합니다. 음이항 회귀 모델을 고려해야 합니다.


In [12]:
# 음이항 회귀 모델 적합
negbin_model = smf.glm(formula='EPDO_cnt ~ bus_station_cnt + traffic_cnt + pleasure_cnt + seoul_police_cnt + park_cnt + crossroad_cnt + 도로10이하_cnt + 도로10_20_cnt + 도로20_30_cnt + 도로30이상_cnt + child_cnt + silver_cnt + crosswalk_cnt', 
                       data=data, 
                       family=sm.families.NegativeBinomial()).fit()

print(negbin_model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:               EPDO_cnt   No. Observations:                 1200
Model:                            GLM   Df Residuals:                     1186
Model Family:        NegativeBinomial   Df Model:                           13
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2921.0
Date:                Thu, 11 Jul 2024   Deviance:                       4139.0
Time:                        09:48:51   Pearson chi2:                 1.20e+04
No. Iterations:                    18   Pseudo R-squ. (CS):             0.6154
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.8133      0.106  

c:\Users\inho0\Downloads\confer\tobigs\lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Poisson 회귀 모델
poisson_model = smf.glm(formula='EPDO_cnt ~ bus_station_cnt + traffic_cnt + pleasure_cnt + seoul_police_cnt + park_cnt + crossroad_cnt + 도로10이하_cnt + 도로10_20_cnt + 도로20_30_cnt + 도로30이상_cnt + child_cnt + silver_cnt + crosswalk_cnt', 
                       data=data, 
                       family=sm.families.Poisson()).fit()

# 음이항 회귀 모델
negbin_model = smf.glm(formula='EPDO_cnt ~ bus_station_cnt + traffic_cnt + pleasure_cnt + seoul_police_cnt + park_cnt + crossroad_cnt + 도로10이하_cnt + 도로10_20_cnt + 도로20_30_cnt + 도로30이상_cnt + child_cnt + silver_cnt + crosswalk_cnt', 
                       data=data, 
                       family=sm.families.NegativeBinomial()).fit()

poisson_pred = poisson_model.predict(data)
negbin_pred = negbin_model.predict(data)

actual = data['EPDO_cnt']

# RMSE 
poisson_rmse = np.sqrt(mean_squared_error(actual, poisson_pred))
negbin_rmse = np.sqrt(mean_squared_error(actual, negbin_pred))

# MAE 
poisson_mae = mean_absolute_error(actual, poisson_pred)
negbin_mae = mean_absolute_error(actual, negbin_pred)

# R-squared 
poisson_r2 = r2_score(actual, poisson_pred)
negbin_r2 = r2_score(actual, negbin_pred)

print("Poisson 회귀 모델")
print("RMSE:", poisson_rmse)
print("MAE:", poisson_mae)
print("R-squared:", poisson_r2)

print("\n음이항 회귀 모델")
print("RMSE:", negbin_rmse)
print("MAE:", negbin_mae)
print("R-squared:", negbin_r2)


Poisson 회귀 모델
RMSE: 19.43764206731411
MAE: 9.102155182221498
R-squared: 0.08201676139491265

음이항 회귀 모델
RMSE: 29.050833783212575
MAE: 10.711443230961368
R-squared: -1.0505237006951123


c:\Users\inho0\Downloads\confer\tobigs\lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [16]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt

poisson_residuals = poisson_model.resid_response

negbin_residuals = negbin_model.resid_response

# 잔차 히스토그램 시각화
def plot_residuals_histogram(residuals, title):
    hist_data = [residuals]
    group_labels = ['Residuals']
    fig = ff.create_distplot(hist_data, group_labels, bin_size=1, show_rug=False)
    fig.update_layout(title=title, xaxis_title='Residuals', yaxis_title='Density')
    fig.show()

plot_residuals_histogram(poisson_residuals, 'Poisson Model Residuals Distribution')

plot_residuals_histogram(negbin_residuals, 'Negative Binomial Model Residuals Distribution')


In [20]:
# 음이항 회귀 모델의 alpha 파라미터 추정
alpha = negbin_model.scale
print("Negative Binomial Model alpha (dispersion parameter):", alpha)

Negative Binomial Model alpha (dispersion parameter): 1.0
